In [12]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os

In [13]:
api_key = os.environ["MISTRAL_API_KEY"]
print(os.getenv("MISTRAL_API_KEY"))

n8zOo7FKHGqM8Jebj7KKTEYGtDaOwF0X


In [14]:
client = Mistral(api_key=api_key)

In [23]:
#import txt.file
with open('/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/remark.txt') as f:
    text = f.read()

print(text)

                                                                                                                                                           M/U clean-out BHA
                                                                                                                            Removed master bushing and installed PS-21 slips
                                                                                                  RIH w/ clean-out BHA from 67 mMD  to 806 mMD. Avg. running speed 740 m/hr.
                                                     Filled pipe and staged up pumps to 3100 lpm/36 bar. OBM in returns, got a gas peak of 18% and 8ppm H2S in return fluid.
                                                                                                                                        Prepared for R/U wireline equipment.
                                                                                                                      Performed space o

In [24]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

120

In [25]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [26]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [47]:
question = "Summarize in one sentence the main findings of the report."
question_embeddings = np.array([get_text_embedding(question)])

In [48]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [49]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [50]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)

'The report details various drilling operations, including hole drilling at specific depths and rates, equipment changes, safety procedures, and investigations into equipment failures.'